In [2]:
import torch
from diffusers import LTXPipeline, AutoModel
from diffusers.hooks import apply_group_offloading
from diffusers.utils import export_to_video

In [3]:
# fp8 layerwise weight-casting
transformer = AutoModel.from_pretrained(
    "Lightricks/LTX-Video",
    subfolder="transformer",
    torch_dtype=torch.bfloat16
)
transformer.enable_layerwise_casting(
    storage_dtype=torch.float8_e4m3fn, compute_dtype=torch.bfloat16
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.45it/s]


In [4]:
from diffusers import LTXPipeline

pipeline = LTXPipeline.from_pretrained(
    "Lightricks/LTX-Video",
    torch_dtype=torch.bfloat16  # 또는 torch.float16
)

Loading pipeline components...: 100%|██████████| 5/5 [00:06<00:00,  1.20s/it]


In [5]:
# group-offloading
onload_device = torch.device("cuda")
offload_device = torch.device("cpu")
pipeline.transformer.enable_group_offload(onload_device=onload_device, offload_device=offload_device, offload_type="leaf_level", use_stream=True)
apply_group_offloading(pipeline.text_encoder, onload_device=onload_device, offload_type="block_level", num_blocks_per_group=2)
apply_group_offloading(pipeline.vae, onload_device=onload_device, offload_type="leaf_level")

prompt = (
    "A male student with glasses is sleeping with his head on the desk. "
    "Suddenly, he experiences a small twitch, like a seizure. "
    "He wakes up abruptly and slowly gets up from his seat."
)

negative_prompt = "low quality, blurry, distorted face, disfigured, bad anatomy, deformed"

In [6]:
video = pipeline(
    prompt=prompt,
    negative_prompt=negative_prompt,
    width=768,
    height=512,
    num_frames=90,
    decode_timestep=0.03,
    decode_noise_scale=0.03,
    num_inference_steps=25,
).frames[0]

100%|██████████| 25/25 [14:58<00:00, 35.95s/it]


In [20]:
export_to_video(video, "output.mp4", fps=30)

It is recommended to use `export_to_video` with `imageio` and `imageio-ffmpeg` as a backend. 
These libraries are not present in your environment. Attempting to use legacy OpenCV backend to export video. 
Support for the OpenCV backend will be deprecated in a future Diffusers version


ImportError: 
export_to_video requires the OpenCV library but it was not found in your environment. You can install it with pip: `pip
install opencv-python`
